In [1]:
%pip install tensorflow pyDOE


  Preparing metadata (setup.py) ... done
  Created wheel for pyDOE: filename=pyDOE-0.3.8-py3-none-any.whl size=18170 sha256=e903923a955de769678b0264f772d4a6d6481ddeca72e6c978bcc59c1d788c20
  Stored in directory: /root/.cache/pip/wheels/96/b9/5d/1138ea8c8f212bce6e97ae58847b7cc323145b3277f2129e2b
Successfully built pyDOE


In [11]:
import tensorflow as tf
import numpy as np
import scipy.io
from pyDOE import lhs
import time
from scipy.optimize import minimize

# Configuración
tf.random.set_seed(1234)
np.random.seed(1234)

# Check GPU availability
print("GPU Available:", tf.config.list_physical_devices('GPU'))

# Load data - NOTA: Asegúrate de que los nombres coincidan con tu archivo .mat
data = scipy.io.loadmat('NLS.mat')
t = data['tt'].flatten()[:, None]  # Ajusta según tu archivo
x = data['x'].flatten()[:, None]  # Ajusta según tu archivo
Exact = data['uu']  # Ajusta según tu archivo

Exact_u = np.real(Exact)
Exact_v = np.imag(Exact)
Exact_h = np.sqrt(Exact_u**2 + Exact_v**2)

X, T = np.meshgrid(x, t)
X_star = np.hstack((X.flatten()[:, None], T.flatten()[:, None]))
u_star = Exact_u.T.flatten()[:, None]
v_star = Exact_v.T.flatten()[:, None]
h_star = Exact_h.T.flatten()[:, None]

# Domain bounds
lb = np.array([-5.0, 0.0])
ub = np.array([5.0, np.pi/2])

N0 = 100
N_b = 100
N_f = 20000
layers = [2, 40, 40, 40, 2]

# Initial conditions
idx_x = np.random.choice(x.shape[0], N0, replace=False)
x0 = x[idx_x, :]
u0 = Exact_u[idx_x, 0:1]
v0 = Exact_v[idx_x, 0:1]

# Boundary conditions
idx_t = np.random.choice(t.shape[0], N_b, replace=False)
tb = t[idx_t, :]

# Collocation points
X_f = lb + (ub - lb) * lhs(2, N_f)

# Training points
X0 = np.concatenate((x0, 0 * x0), 1)
X_lb = np.concatenate((0 * tb + lb[0], tb), 1)
X_ub = np.concatenate((0 * tb + ub[0], tb), 1)
X_u_train = np.vstack([X0, X_lb, X_ub])

class PhysicsInformedNN(tf.keras.Model):
    def __init__(self, layers, lb, ub):
        super().__init__()
        self.lb = tf.constant(lb, dtype=tf.float32)
        self.ub = tf.constant(ub, dtype=tf.float32)

        # Construir red neuronal
        self.hidden_layers = []
        for i in range(len(layers)-2):
            self.hidden_layers.append(
                tf.keras.layers.Dense(layers[i+1],
                                    activation='tanh',
                                    kernel_initializer='glorot_normal')
            )
        self.output_layer = tf.keras.layers.Dense(layers[-1])

    def call(self, X):
        # Normalizar entrada
        H = 2.0 * (X - self.lb) / (self.ub - self.lb) - 1.0

        # Pasar por capas ocultas
        for layer in self.hidden_layers:
            H = layer(H)

        # Capa de salida
        return self.output_layer(H)

    def net_uv(self, x, t):
        X = tf.concat([x, t], axis=1)
        uv = self(X)
        u = uv[:, 0:1]
        v = uv[:, 1:2]
        return u, v

    def net_uv_with_gradients(self, x, t):
        with tf.GradientTape(persistent=True) as tape2:
            tape2.watch(x)
            tape2.watch(t)
            u, v = self.net_uv(x, t)

        u_x = tape2.gradient(u, x)
        v_x = tape2.gradient(v, x)

        del tape2
        return u, v, u_x, v_x

    def net_f_uv(self, x, t):
        with tf.GradientTape(persistent=True) as tape1:
            tape1.watch(x)
            tape1.watch(t)
            u, v, u_x, v_x = self.net_uv_with_gradients(x, t)

        u_t = tape1.gradient(u, t)
        v_t = tape1.gradient(v, t)
        u_xx = tape1.gradient(u_x, x)
        v_xx = tape1.gradient(v_x, x)

        del tape1

        # Ecuación de Schrödinger no lineal
        f_u = u_t + 0.5 * v_xx + (u**2 + v**2) * v
        f_v = v_t - 0.5 * u_xx - (u**2 + v**2) * u

        return f_u, f_v

# Convertir datos a tensores
X0_tf = tf.convert_to_tensor(X0, dtype=tf.float32)
X_lb_tf = tf.convert_to_tensor(X_lb, dtype=tf.float32)
X_ub_tf = tf.convert_to_tensor(X_ub, dtype=tf.float32)
X_f_tf = tf.convert_to_tensor(X_f, dtype=tf.float32)
u0_tf = tf.convert_to_tensor(u0, dtype=tf.float32)
v0_tf = tf.convert_to_tensor(v0, dtype=tf.float32)

# Inicializar modelo
model = PhysicsInformedNN(layers, lb, ub)

# Pase inicial para construir el modelo
_ = model(tf.zeros((1, 2)))

# Optimizador Adam
optimizer_adam = tf.keras.optimizers.Adam(learning_rate=0.001)

@tf.function
def compute_loss():
    # Pérdida de condiciones iniciales
    u0_pred, v0_pred = model.net_uv(X0_tf[:, 0:1], X0_tf[:, 1:2])
    loss_0 = tf.reduce_mean(tf.square(u0_tf - u0_pred)) + \
              tf.reduce_mean(tf.square(v0_tf - v0_pred))

    # Pérdida de condiciones de contorno periódicas
    u_lb_pred, v_lb_pred, u_x_lb_pred, v_x_lb_pred = model.net_uv_with_gradients(
        X_lb_tf[:, 0:1], X_lb_tf[:, 1:2])
    u_ub_pred, v_ub_pred, u_x_ub_pred, v_x_ub_pred = model.net_uv_with_gradients(
        X_ub_tf[:, 0:1], X_ub_tf[:, 1:2])

    loss_b = tf.reduce_mean(tf.square(u_lb_pred - u_ub_pred)) + \
             tf.reduce_mean(tf.square(v_lb_pred - v_ub_pred)) + \
             tf.reduce_mean(tf.square(u_x_lb_pred - u_x_ub_pred)) + \
             tf.reduce_mean(tf.square(v_x_lb_pred - v_x_ub_pred))

    # Pérdida de la ecuación física
    f_u_pred, f_v_pred = model.net_f_uv(X_f_tf[:, 0:1], X_f_tf[:, 1:2])
    loss_f = tf.reduce_mean(tf.square(f_u_pred)) + tf.reduce_mean(tf.square(f_v_pred))

    return loss_0 + loss_b + loss_f

@tf.function
def train_step():
    with tf.GradientTape() as tape:
        loss = compute_loss()
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer_adam.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

# Entrenamiento con Adam
def train_adam(nIter):
    print("Iniciando entrenamiento Adam...")
    for it in range(nIter):
        loss_value = train_step()
        if it % 1000 == 0:
            print(f'Iteración {it}, Pérdida: {loss_value.numpy():.3e}')

# Entrenamiento con L-BFGS
def get_weights():
    return np.concatenate([w.numpy().flatten() for w in model.trainable_variables])

def set_weights(weights):
    shapes = [w.shape for w in model.trainable_variables]
    sizes = [np.prod(shape) for shape in shapes]

    start = 0
    new_weights = []
    for shape, size in zip(shapes, sizes):
        new_weights.append(np.reshape(weights[start:start+size], shape))
        start += size

    for w, new_w in zip(model.trainable_variables, new_weights):
        w.assign(new_w)

def loss_and_grad(weights):
    set_weights(weights)
    with tf.GradientTape() as tape:
        loss_value = compute_loss()
    grads = tape.gradient(loss_value, model.trainable_variables)
    grad_flat = np.concatenate([g.numpy().flatten() for g in grads])
    return loss_value.numpy().astype(np.float64), grad_flat.astype(np.float64)

# Entrenamiento principal
start_time = time.time()

# Fase Adam
train_adam(10000)

# Fase L-BFGS
print("Iniciando entrenamiento L-BFGS...")
initial_weights = get_weights()
result = minimize(loss_and_grad, initial_weights,
                  method='L-BFGS-B', jac=True,
                  options={'maxiter': 50000, 'maxfun': 50000,
                          'ftol': 1.0 * np.finfo(float).eps,
                          'gtol': 1.0 * np.finfo(float).eps})

set_weights(result.x)

elapsed = time.time() - start_time
print(f'Tiempo de entrenamiento total: {elapsed:.2f} segundos')

# Predicción y evaluación
X_star_tf = tf.convert_to_tensor(X_star, dtype=tf.float32)
u_pred, v_pred = model.net_uv(X_star_tf[:, 0:1], X_star_tf[:, 1:2])
h_pred = tf.sqrt(u_pred**2 + v_pred**2)

error_u = np.linalg.norm(u_star - u_pred.numpy(), 2) / np.linalg.norm(u_star, 2)
error_v = np.linalg.norm(v_star - v_pred.numpy(), 2) / np.linalg.norm(v_star, 2)
error_h = np.linalg.norm(h_star - h_pred.numpy(), 2) / np.linalg.norm(h_star, 2)

print(f'Error u: {error_u:.3e}')
print(f'Error v: {error_v:.3e}')
print(f'Error h: {error_h:.3e}')

GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Iniciando entrenamiento Adam...
Iteración 0, Pérdida: 1.240e+00
Iteración 1000, Pérdida: 4.853e-02
Iteración 2000, Pérdida: 3.055e-02
Iteración 3000, Pérdida: 2.191e-02
Iteración 4000, Pérdida: 1.555e-02
Iteración 5000, Pérdida: 1.236e-02
Iteración 6000, Pérdida: 1.453e-02
Iteración 7000, Pérdida: 8.785e-03
Iteración 8000, Pérdida: 7.471e-03
Iteración 9000, Pérdida: 1.351e-02
Iniciando entrenamiento L-BFGS...
Tiempo de entrenamiento total: 484.16 segundos
Error u: 3.095e-02
Error v: 5.064e-02
Error h: 7.719e-03
